In [1]:
import pandas as pd
import numpy as np
from Merge import *
from sklearn.cross_validation import train_test_split
from model_builder import *
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/linda/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#load data
yX_train= pd.read_csv('train_All2.csv')

test= pd.read_csv('test_All2.csv') 



/Users/linda/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
yX_train['is_install'] = yX_train['timestamp_at_install'].notnull() * 1

In [4]:
yX_train['time_delta'] = pd.to_datetime(yX_train['time_of_this_impression'])-pd.to_datetime(yX_train['created_at'])
yX_train['time_delta'] = yX_train['time_delta'].dt.days
test['time_delta'] = pd.to_datetime(test['time_of_this_impression'])-pd.to_datetime(test['created_at'])
test['time_delta'] = test['time_delta'].dt.days

In [43]:
yX_train.columns

Index([u'advertiser_app_store_id', u'country_code', u'city', u'campaign_id',
       u'creative_id', u'device_language', u'device_make', u'device_model',
       u'device_platform', u'device_connection', u'device_os_version',
       u'device_screen_height', u'device_screen_width', u'device_volume',
       u'event_id', u'n_campaign_views', u'n_vungle_installs',
       u'publisher_app_store_id', u'time_of_last_delivery_this_campaign',
       u'time_of_last_delivery_this_creative',
       u'time_of_last_delivery_any_installed_app',
       u'time_of_last_vungle_delivery', u'time_of_this_impression',
       u'timestamp_at_install', u'time_of_this_request', u'time_zone',
       u'market_id', u'is_publisher', u'platform', u'n_user_ratings',
       u'content_rating', u'price_value', u'n_min_downloads',
       u'n_max_downloads', u'primary_category', u'created_at',
       u'application_id', u'video_id', u'is_streaming',
       u'enforce_language_match', u'delivery_frequency_cap',
       u'time_to

In [5]:
test.loc[~test.time_of_last_delivery_any_installed_app.isnull(),'time_of_last_delivery_any_installed_app']='t'
test.time_of_last_delivery_any_installed_app=test.time_of_last_delivery_any_installed_app.fillna('f')
yX_train.loc[~yX_train.time_of_last_delivery_any_installed_app.isnull(),'time_of_last_delivery_any_installed_app']='t'
yX_train.time_of_last_delivery_any_installed_app=yX_train.time_of_last_delivery_any_installed_app.fillna('f')

In [6]:
def model_selection_fun(train,test,mod,interactions):
    
    #cont_features_pc = []
    cat_interactions_pc = interactions
    cat_features_pc =mod
    X_train, y_train, X_test = train, train.pop('is_install'), test

    feature_creator = FeatureCreator()
    design_matrix_transformer = FeatureHasher(22, cat_features_pc, None, cat_interactions_pc, store_fmap=True) # You can experiment with hasher bits (we used 18 here)

    X_test = feature_creator.transform(X_test, inplace=True)
    X_test, f_map = design_matrix_transformer.fit_transform(X_test)

    X_train = feature_creator.transform(X_train, inplace=True)
    X_train, f_map = design_matrix_transformer.fit_transform(X_train)

    logistic_baseline = logistic_model_final(X_train, y_train,0.05)

    # Calculate prediction/probability of train and test
    X_train_predictions = logistic_baseline.predict(X_train)
    X_train_predprob = logistic_baseline.predict_proba(X_train)[:, 1]


    X_test_predprob = logistic_baseline.predict_proba(X_test)[:, 1]


    # Calculate metrics of train, validation and test set.
    #lr_ll_val = -logistic_baseline.best_score_

    lr_ll_train = log_loss(y_train, X_train_predprob)
    lr_auc_train = roc_auc_score(y_train, X_train_predprob)

    
    # Print out the results
    #print "Best parameter: ", logistic_baseline.best_params_

    #print "Log Loss (Validation): %f" % lr_ll_val

    print "Log Loss (Train): %f" % lr_ll_train
    print "AUC (Train): %f" % lr_auc_train
    return X_train_predprob, X_test_predprob 


In [7]:
def logistic_model_final(X_train, y_train,c=0.1):
    """
    use crossvalidation (CV) to report the best parameter 'C'
    parameter C: Inverse of regularization strength; must be a positive float. 
    Check LogisticRegression() in sklearn for more information
    """
    print('Train Regression Model')

    model=LogisticRegression(C=c,penalty='l2',solver='sag',n_jobs=-1)

    model.fit(X_train, y_train)
    return model


In [44]:
base_model =['advertiser_app_store_id', 'country_code', 'publisher_app_store_id',u'n_campaign_views',
       u'device_model', u'n_vungle_installs', u'device_connection', u'device_os_version',u'time_of_last_delivery_this_campaign']

interaction=[('device_model', 'publisher_app_store_id'), ('device_model', 'country_code'), 
('device_model', 'ever_install_flag'), ('device_model', 'device_os_version'), 
('device_model', 'device_connection'), ('device_model', 'advertiser_app_store_id'),
('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'), 
('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', 'country_code'), 
('device_connection', 'device_os_version'), ('publisher_app_store_id', 'country_code'), 
('advertiser_app_store_id', 'device_os_version'), ('publisher_app_store_id', 'device_os_version'),
(u'device_connection', 'n_vungle_installs'), ('device_connection','time_of_last_delivery_this_campaign'),
('country_code', 'device_os_version'), ('n_campaign_views', 'n_vungle_installs')]


yX = yX_train[['is_install']+base_model].copy()
yY = test[base_model].copy()
b = convert_to_unknown(yX,base_model)
b = convert_to_string(b,base_model)
a = convert_to_unknown(yY,base_model)
a = convert_to_string(a,base_model)
print '------base model-------'
print base_model
print '------interaction-----'
print interaction
y_pred_train,y_pred= model_selection_fun(b,a,base_model,interaction)

------base model-------
['advertiser_app_store_id', 'country_code', 'publisher_app_store_id', u'time_zone', u'market_id', u'is_publisher', u'platform', u'n_user_ratings', u'content_rating']
------interaction-----
[('advertiser_app_store_id', 'publisher_app_store_id', 'country_code'), ('advertiser_app_store_id', 'publisher_app_store_id'), ('advertiser_app_store_id', 'country_code')]
Train Regression Model
Log Loss (Train): 0.380786
AUC (Train): 0.793073


In [45]:
df = pd.DataFrame(y_pred)
df_train = pd.DataFrame(y_pred_train)


In [46]:
df['row'] =test.row
df_train['y'] = yX_train.is_install

In [47]:
df.columns=['y','row']
df_train.columns = ['prob','y']
df=df[['row','y']]

In [49]:
df.to_csv('prediction_0307_3.csv',index=False)
df_train.to_csv('y_trainprediction_0307_3.csv',index=False)

In [16]:
df=df[['row','y']]

In [30]:
df.head()

,y,row
0,0.476072,1
1,0.080354,2
2,0.179363,3
3,0.105121,4
4,0.105527,5


In [31]:
df_train.head()


,prob,y
0,0.596533,0
1,0.059309,0
2,0.021290,0
3,0.187056,0
4,0.042719,0
